In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv')
print(f'Orginal shape: {df.shape}')
df.head()

df_clean = df.copy()

Orginal shape: (7043, 21)


In [2]:
df_clean

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [3]:
df_clean.columns = df_clean.columns.str.replace(r'([a-z])([A-Z])', r'\1_\2', regex=True).str.lower()
df_clean.columns

Index(['customer_id', 'gender', 'senior_citizen', 'partner', 'dependents',
       'tenure', 'phone_service', 'multiple_lines', 'internet_service',
       'online_security', 'online_backup', 'device_protection', 'tech_support',
       'streaming_tv', 'streaming_movies', 'contract', 'paperless_billing',
       'payment_method', 'monthly_charges', 'total_charges', 'churn'],
      dtype='object')

In [4]:
df_clean['total_charges'] = pd.to_numeric(df_clean['total_charges'], errors='coerce').fillna(0)

print(df_clean.dtypes)
print(df_clean.loc[df_clean['total_charges'] == 0, 'total_charges'].count())

customer_id           object
gender                object
senior_citizen         int64
partner               object
dependents            object
tenure                 int64
phone_service         object
multiple_lines        object
internet_service      object
online_security       object
online_backup         object
device_protection     object
tech_support          object
streaming_tv          object
streaming_movies      object
contract              object
paperless_billing     object
payment_method        object
monthly_charges      float64
total_charges        float64
churn                 object
dtype: object
11


In [5]:
df_clean['expected_total_charges'] = df_clean['tenure'] * df_clean['monthly_charges']
df_clean['charges_difference'] = df_clean['total_charges'] - df_clean['expected_total_charges']

conditions = [
    df_clean['charges_difference'] < -50,
    df_clean['charges_difference'] < -10,
    df_clean['charges_difference'] <= 10,
    df_clean['charges_difference']<= 50,
    df_clean['charges_difference'] > 50
]

choices = ['significant_discount', 'minor_discount', 'as_expected', 'minor_premium', 'significant_premium']

df_clean['pricing_pattern'] = np.select(conditions, choices, default='as_expected')

df_clean.head()


,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,...,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn,expected_total_charges,charges_difference,pricing_pattern
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,29.85,0.00,as_expected
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,One year,No,Mailed check,56.95,1889.50,No,1936.30,-46.80,minor_discount
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,107.70,0.45,as_expected
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,1903.50,-62.75,significant_discount
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,141.40,10.25,minor_premium


In [6]:
column_order = [
    # Identifiers
    'customer_id',

    # Demographics
    'gender',
    'senior_citizen',
    'partner',
    'dependents',

    # Account info
    'tenure',
    'contract',

    # Services - Phone
    'phone_service',
    'multiple_lines',
    
    # Services - Internet
    'internet_service',
    'online_security',
    'online_backup',
    'device_protection',
    'tech_support',
    'streaming_tv',
    'streaming_movies',
    
    # Billing
    'paperless_billing',
    'payment_method',
    'monthly_charges',
    'total_charges',
    'expected_total_charges',
    'charges_difference',
    'pricing_pattern',
    
    # Target
    'churn'
]

df_clean = df_clean[column_order]
df_clean.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,contract,phone_service,multiple_lines,internet_service,...,streaming_tv,streaming_movies,paperless_billing,payment_method,monthly_charges,total_charges,expected_total_charges,charges_difference,pricing_pattern,churn
0,7590-VHVEG,Female,0,Yes,No,1,Month-to-month,No,No phone service,DSL,...,No,No,Yes,Electronic check,29.85,29.85,29.85,0.00,as_expected,No
1,5575-GNVDE,Male,0,No,No,34,One year,Yes,No,DSL,...,No,No,No,Mailed check,56.95,1889.50,1936.30,-46.80,minor_discount,No
2,3668-QPYBK,Male,0,No,No,2,Month-to-month,Yes,No,DSL,...,No,No,Yes,Mailed check,53.85,108.15,107.70,0.45,as_expected,Yes
3,7795-CFOCW,Male,0,No,No,45,One year,No,No phone service,DSL,...,No,No,No,Bank transfer (automatic),42.30,1840.75,1903.50,-62.75,significant_discount,No
4,9237-HQITU,Female,0,No,No,2,Month-to-month,Yes,No,Fiber optic,...,No,No,Yes,Electronic check,70.70,151.65,141.40,10.25,minor_premium,Yes


In [7]:
unnormalized_columns = ['multiple_lines', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies']

df_clean[unnormalized_columns] = df_clean[unnormalized_columns].replace({'No internet service': 'No', 'No phone service': 'No'})

for col in unnormalized_columns:
    print(df_clean[col].value_counts())

multiple_lines
No     4072
Yes    2971
Name: count, dtype: int64
online_security
No     5024
Yes    2019
Name: count, dtype: int64
online_backup
No     4614
Yes    2429
Name: count, dtype: int64
device_protection
No     4621
Yes    2422
Name: count, dtype: int64
tech_support
No     4999
Yes    2044
Name: count, dtype: int64
streaming_tv
No     4336
Yes    2707
Name: count, dtype: int64
streaming_movies
No     4311
Yes    2732
Name: count, dtype: int64


In [8]:
binary_columns = [col for col in df_clean.columns
                  if df_clean[col].nunique() == 2
                  and col not in ['gender', 'senior_citizen']]

for col in binary_columns:
    df_clean[col] = df_clean[col].map({'Yes': True, 'No': False})
    print(df_clean[col].value_counts())

print(df_clean[binary_columns].dtypes)

partner
False    3641
True     3402
Name: count, dtype: int64
dependents
False    4933
True     2110
Name: count, dtype: int64
phone_service
True     6361
False     682
Name: count, dtype: int64
multiple_lines
False    4072
True     2971
Name: count, dtype: int64
online_security
False    5024
True     2019
Name: count, dtype: int64
online_backup
False    4614
True     2429
Name: count, dtype: int64
device_protection
False    4621
True     2422
Name: count, dtype: int64
tech_support
False    4999
True     2044
Name: count, dtype: int64
streaming_tv
False    4336
True     2707
Name: count, dtype: int64
streaming_movies
False    4311
True     2732
Name: count, dtype: int64
paperless_billing
True     4171
False    2872
Name: count, dtype: int64
churn
False    5174
True     1869
Name: count, dtype: int64
partner              bool
dependents           bool
phone_service        bool
multiple_lines       bool
online_security      bool
online_backup        bool
device_protection    bool
tech_su

In [11]:
df_clean['senior_citizen'] = df_clean['senior_citizen'].astype(bool)
print(df_clean.dtypes)
print(df_clean.head(10))

customer_id                object
gender                     object
senior_citizen               bool
partner                      bool
dependents                   bool
tenure                      int64
contract                   object
phone_service                bool
multiple_lines               bool
internet_service           object
online_security              bool
online_backup                bool
device_protection            bool
tech_support                 bool
streaming_tv                 bool
streaming_movies             bool
paperless_billing            bool
payment_method             object
monthly_charges           float64
total_charges             float64
expected_total_charges    float64
charges_difference        float64
pricing_pattern            object
churn                        bool
dtype: object
  customer_id  gender  senior_citizen  partner  dependents  tenure  \
0  7590-VHVEG  Female           False     True       False       1   
1  5575-GNVDE    Male         